# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [160]:
# Dependencies and Setup
import pandas as pd
import requests
import cartopy.crs as ccrs
import hvplot.pandas
import chardet

# Import API key
from api_keys import geoapify_key

In [161]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy//cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bluff,-46.6000,168.3333,18.85,74,17,4.67,NZ,1675127029
1,1,ushuaia,-54.8000,-68.3000,5.03,87,92,1.55,AR,1675127020
2,2,aklavik,68.2191,-135.0107,-34.68,100,22,0.78,CA,1675127140
3,3,punta arenas,-53.1500,-70.9167,10.62,50,83,7.52,CL,1675126962
4,4,vanimo,-2.6741,141.3028,28.53,77,62,2.40,PG,1675127141


### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [162]:
# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat',size='Humidity',c='City',cmap='rainbow',fill_alpha=0.5,color='City', colorbar=True, geo=True, tiles='OSM',hover_cols=['City'])



In [163]:

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [166]:
ideal_weather_df = city_data_df[(city_data_df['Max Temp'] >= 27) & (city_data_df['Cloudiness'] <= 50) & (city_data_df['Humidity'] >= 65) ]
ideal_weather_df.head()

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,tautira,-17.7333,-149.1500,27.38,75,22,6.45,PF,1675127160
142,142,lorengau,-2.0226,147.2712,28.58,74,40,2.48,PG,1675127212
160,160,samarai,-10.6167,150.6667,29.37,69,21,3.82,PG,1675127218
177,177,san andres,12.5847,-81.7006,27.01,74,20,5.66,CO,1675127228
256,256,bairiki,1.3292,172.9752,30.00,70,20,4.12,KI,1675127277


### Step 3: Create a new DataFrame called `hotel_df`.

In [167]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df.copy()
hotel_df = hotel_df[['City','Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
60,tautira,PF,-17.7333,-149.1500,75,
142,lorengau,PG,-2.0226,147.2712,74,
160,samarai,PG,-10.6167,150.6667,69,
177,san andres,CO,12.5847,-81.7006,74,
256,bairiki,KI,1.3292,172.9752,70,
337,malayal,PH,7.2080,121.9420,78,
403,puerto escondido,MX,15.8500,-97.0667,69,
419,faanui,PF,-16.4833,-151.7500,73,
445,vaitape,PF,-16.5167,-151.7500,73,
515,noumea,NC,-22.2763,166.4572,70,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [168]:
# Set parameters to search for a hotel
radius = 15000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":30
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting hotel search
tautira - nearest hotel: Omati Lodge
lorengau - nearest hotel: Seeadler Bay Hotel
samarai - nearest hotel: No hotel found
san andres - nearest hotel: Hotel Decameron Isleño
bairiki - nearest hotel: Marry's Motel
malayal - nearest hotel: No hotel found
puerto escondido - nearest hotel: Hotel Barlovento
faanui - nearest hotel: Oa Oa Lodge
vaitape - nearest hotel: Oa Oa Lodge
noumea - nearest hotel: Hotel Gondwana
port blair - nearest hotel: Andaman Ocean Breeze


,City,Country,Lat,Lng,Humidity,Hotel Name
60,tautira,PF,-17.7333,-149.1500,75,Omati Lodge
142,lorengau,PG,-2.0226,147.2712,74,Seeadler Bay Hotel
160,samarai,PG,-10.6167,150.6667,69,No hotel found
177,san andres,CO,12.5847,-81.7006,74,Hotel Decameron Isleño
256,bairiki,KI,1.3292,172.9752,70,Marry's Motel
337,malayal,PH,7.2080,121.9420,78,No hotel found
403,puerto escondido,MX,15.8500,-97.0667,69,Hotel Barlovento
419,faanui,PF,-16.4833,-151.7500,73,Oa Oa Lodge
445,vaitape,PF,-16.5167,-151.7500,73,Oa Oa Lodge
515,noumea,NC,-22.2763,166.4572,70,Hotel Gondwana


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [169]:
# Configure the map plot
hotel_plot = hotel_df.hvplot.points('Lng', 'Lat',size='Humidity',c='City',cmap='rainbow',fill_alpha=0.5,color='City', colorbar=True, geo=True, tiles='OSM',hover_cols=['City','Hotel Name','Country'],width=1000, height=500)

In [170]:
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)